In [1]:
import numpy as np
import os, requests
from time import process_time as timer
from PFL_memory_supplements import *
import gc


# Python for Lunch (PFL) - Performance Sessions - C-bindings 

# _A Python on Steroids_ 

Last week, we looked at memory consumption and speeding-up I/O-bound programs in Python. This week, we focus on the complementary case: speeding-up compute-bound programs in Python! (i.e. the actual **fun** stuff!) 

## Motivation

In PFL 19, we looked at time measurements. At the closure, we measure *compute time*, *I/O time* and the *compute-to-I/O ratio*.
We learned that the latter ratio indicates what we need to prioritize when increasing the performance of a program, and also said that compute-enhancing techniques (i.e. *Python-on-Steriods*) only makes sense if that ratio is *considerably higher than 1*.

Measuring those timings in actual, large Python programs is quite difficult due to the high conceptual level of Python.
Realistic programs in Physics are often I/O-bound (i.e. ratio less than 1), cause data are stored in huge files and data copies are needed to hold previous-iteration values.
Some codes (e.g. CESM or CMEMS themselves) are based on finite-difference *ordinary differential equations (ODE's)*, which require only limited external data input.
Those codes are usually *compute-bound*, meaning that the performance bottleneck is the amount of *floating-point operations per second (FLOPS)* that are performed. It is also the case where using Python (as a programming language) can be _**the**_ major limitation.
Today, we look into ways to speed up compute-bound programs in Python, continuing the theme of last week, where we looked at speeding-up I/O-bound programs.

### Programming Language Speed & Python

Not all programming languages are the same. Every programming language has its preferred application scenarios, its intended use and its advantages.
For some (or: most) of you, *Python* is the only programming language you know, whereas the *older generation* can still remember coding in *FORTRAN*.
Those programming languages are very prominent in Physics. When comparing the two, one can overall observe that FORTRAN programs are very, very fast. The drawback of FORTRAN is that its very laborious to program, and very error-prone.
Those are the drawbacks Python was uniquely designed to address and remedy - though the easy of development came at the expense of computation speed.

One point here: if we talk about *Python* as a language, we talk about *plain Python*, not: *NumPy*, *SciPy*, *Scikits*. Those difference we will discuss and experience within this PFL session.
Actuual, plain Python consists of the following instructions:

- for-loops (i.e. `for ... in range(start, stop)`)
- while-loops
- list-comprehensions (Python-exclusive)
- for-each loops (i.e. `for obj in list`)
- lambda-expressions

In the *Computer Sciences (CS)*, the prevalent modern programming language that are most commonly used are:

- Java (e.g. business-logic programs)
- JavaScript (i.e. web-apps)
- C++ (anything but web)
- C (i.e. close-to-chip programming)
- C#
- Python (getting more promiment due to simplicity and AI/ML/DeepLearning-stuff)

In essence, all *high-level languages (HLL's)* - those are: all languages beyond *Assembler*, *C*, *C++/Objective C* and *FORTRAN* - are rooted in **C/C++**. 
That is because those 2 languages translate (via a *compiler*) directly into byte-code that can be executed by the hardware (i.e. the CPU).
This translation step from an easy-to-use, type-less, error-resilient, interpreter-bound HLL into C-code instructions is a *speed malus* (i.e. performance cost) you 'pay' with each function call.

So: using Python costs us speed in computation, and this cost can be measured in the runtime of programs.
One intensely-discussed question in this context is: *How high is that cost ? How much slower is Python, compared to C or C++ ?*
To quantify that, a good starting point are programming language benchmarks & speed comparisons, such as:

- overview: [the computer language benchmark game](https://benchmarksgame-team.pages.debian.net/benchmarksgame/)
- [Python3 vs. C (on Linux) benchmarks](https://benchmarksgame-team.pages.debian.net/benchmarksgame/fastest/python3-gcc.html)
 
What we see in those comparisons is that Python3's speed (as runtime) compared to C is a between 2:1 (Python's runtime is 2x as much as 1 C-runtime) for toy-computations, 
40:1 for frequently occuring compute problems (e.g. sorting) and >100:1 for complex problems (e.g. nBody). Thus, if we want to get **real speed* of our *actual* calculations, we should not do them inside Python.
The logical follow-up question(s):
 
- **What shall we do ?**
- **Is there the possibility to keep using _Python_, but then 'out-source' the calculations to C or C++ ?**

Today, those are the questions we urge to answer!

## Case Study - k-Nearest Neighbour on Particles

In order to answer this question, we design a suitable case study again. In the case study of today, we re-use the particle advection on flow-fields that we developed in PFL 19.
In this case study, we tracer particles move on a defined flow field (i.e. *advection*) over a set time frame (or: number of iterations).
Today, we expand the calculation by determining the *k-nearest neighbouring (kNN)* particles for *each particle* at *each iteration step*.
This neighbourhood computation is well-known in CS as a very compute-intensive operation, as it involves and _N * (N over N)_ distance calculation, followed
by a _k * N log N_ ordering of those distances. It is a typical computation bottleneck.

Further, it's a good example because:
- there is not just *1 solution* to the problem, and it can be solved in many ways
- it's computational complexity is well-known and well-bound
- thus, speed difference actually come from implementation differences
- if follows the 'simple to code, hard to master' paradigm
- it only consists of for-loops, branch conditionals etc., and thus can be done just using the inherent programming language instructions

Let's start and prepare and run this example first using exclusively Python-code.

In [2]:
def kNN_particles_PythonOnly(simulation, k=5):
    N = len(simulation)
    r = (b-a)
    r = math.sqrt(np.dot(r,r))
    distance_matrix = np.ones((N,N), dtype=np.float64) * r
    for i in range(0, N):
        for j in range (i, N):
            if i==j:
                distance_matrix[i,j] = .0
                continue
            # dv = simulation[i].pt[0:1]-simulation[j].pt[0:1]
            # dv_len = dv[0]*dv[0]+dv[1]*dv[1]
            dx = simulation[i].pt[0]-simulation[j].pt[0]
            dy = simulation[i].pt[1]-simulation[j].pt[1]
            dv_len_sqr = dx*dx+dy*dy
            distance_matrix[i, j] = dv_len_sqr
            distance_matrix[j, i] = dv_len_sqr
    
    kNN_indices = np.ones((N,k), dtype=np.uint32) * -1
    kNN_distances = np.ones((N,k), dtype=np.float64) * r
    for i in range(0, N):
        for j in range (0, N):
            if i==j:
                continue
            m = 0
            while (m<k) and ((kNN_indices[i, m] < 0) or (kNN_distances[i, m] <= distance_matrix[i, j])):
                m += 1
            if not m<k:
                continue
            l = k-1
            while l>m:
                kNN_indices[i, l] = kNN_indices[i, l-1]
                kNN_distances[i, l] = kNN_distances[i, l-1]
            kNN_indices[i, m] = j
            kNN_distances[i, m] = distance_matrix[i, j]
    return kNN_indices, kNN_distances
            


if not os.path.exists('perlin.nc'):
    data_url = "https://surfdrive.surf.nl/files/index.php/s/T7QyLbGjaGMdnVD/download"
    requests.get(data_url)
# ==== Load flow-field data - measure the time ==== #
s_io_file_time = timer()
f = netcdf.netcdf_file('perlin.nc', 'r')
ftimes = f.variables['time'].data
fX = f.variables['lon'].data
fY = f.variables['lat'].data
fU = f.variables['u'].data
fV = f.variables['v'].data
f.close()
e_io_file_time = timer()
io_file_time = e_io_file_time-s_io_file_time

a = np.array([fX[0], fY[0]])
b = np.array([fX[-1], fY[-1]])
t_0_N = np.array([ftimes[0], ftimes[-1]])
T = t_0_N[1]-t_0_N[0]
dt = 12.0 * 60.0 * 60.0
sim_steps = int(math.floor(T/dt))
total_compute_times = []
advect_compute_time = []
kNN_compute_time = []

sim = Simulation_Benchmark(fX, fY, ftimes, fU, fV)
for i in range(0, 512):
    pt = (b - a) * np.random.random_sample(2)  + a
    sim.add_particle(pt[0], pt[1])

# total_compute_times.clear()
print("==== running the simulation with competing procs ====")
stime_sim = timer()
prev_int_proc = -1
cur_int_proc = 0
while sim.sim_time < sim.sim_end_time:
    stime_compute_advect = timer()
    sim.advect_once(dt)
    etime_compute_advect = timer()
    stime_kNN = timer()
    kNN_particles_PythonOnly(sim, k=5)
    etime_kNN = timer()
    advectTime = etime_compute_advect-stime_compute_advect
    knnTime = etime_kNN-stime_kNN
    advect_compute_time.append(advectTime)
    kNN_compute_time.append(knnTime)
    total_compute_times.append(advectTime+knnTime)
    proc_done = (sim.sim_time / sim.sim_end_time)*100
    cur_int_proc = int(math.floor(proc_done/10))
    if prev_int_proc != cur_int_proc:
        print("Processing at {:10.7f} percent ...".format(proc_done))
    prev_int_proc = cur_int_proc

etime_sim = timer()
sim_total_time = etime_sim-stime_sim
print("Total processing time (high-res timer): {:10.7f} sec.".format(sim_total_time))
avg_ptimer_hr = np.array(total_compute_times).mean()
print("Average processing kernel time (high-res timer): {:10.7f} sec.".format(avg_ptimer_hr))

total_compute_time = np.array(total_compute_times).sum()
total_advect_time = np.array(advect_compute_time).sum()
total_knn_time = np.array(kNN_compute_time).sum()

compute_to_total_ratio = total_compute_time / (sim_total_time+io_file_time)
compute_to_iototal = total_compute_time / (sim.io_mem_time+io_file_time)
io_to_sim_ratio = sim.io_mem_time / (sim.compute_time+sim.io_mem_time)
advect_to_sim = sim.compute_time / (sim.compute_time+sim.io_mem_time)
advect_to_iosim = sim.compute_time / sim.io_mem_time+io_file_time
advect_compute_percentage = (total_advect_time / total_compute_time) * 100.0
knn_compute_percentage = (total_knn_time / total_compute_time) * 100.0

total_compute_time_PythonOnly = total_compute_time
total_advect_time_PythonOnly = total_advect_time
total_knn_time_PythonOnly = total_knn_time

print("Compute percentage of runtime:                       {:10.7f}".format(compute_to_total_ratio*100.0))
print("Compute percentage simulation time (Advection-only): {:10.7f}".format(advect_to_sim*100.0))
print("I/O percentage of simulation time:                   {:10.7f}".format(io_to_sim_ratio*100.0))
print("------------------------------------------------------------------")
print("Ratio Compute time vs. I/O time:              {:10.7f}".format(compute_to_iototal))
print("Ratio Advection time vs. I/O-simulation time: {:10.7f}".format(advect_to_iosim))
print("------------------------------------------------------------------")
print("Percentage of Advection on Compute time:         {:10.4f}".format(advect_compute_percentage))
print("Percentage of Nearest-Neighbour on Compute time: {:10.4f}".format(knn_compute_percentage))
del sim
del ftimes
del fX
del fY
del fU
del fV
del total_compute_times
del advect_compute_time
del kNN_compute_time
gc.collect()

/home/ckehl/.local/lib/python3.6/site-packages/scipy/io/netcdf.py:314: RuntimeWarning: Cannot close a netcdf_file opened with mmap=True, when netcdf_variables or arrays referring to its data still exist. All data arrays obtained from such files refer directly to data on disk, and must be copied before the file can be cleanly closed. (See netcdf_file docstring for more information on mmap.)
  ), category=RuntimeWarning)


==== running the simulation with competing procs ====
Processing at  0.1351351 percent ...
Processing at 10.0000000 percent ...
Processing at 20.0000000 percent ...
Processing at 30.0000000 percent ...
Processing at 40.0000000 percent ...
Processing at 50.0000000 percent ...
Processing at 60.0000000 percent ...
Processing at 70.0000000 percent ...
Processing at 80.0000000 percent ...
Processing at 90.0000000 percent ...
Processing at 100.0000000 percent ...
Total processing time (high-res timer): 631.2895284 sec.
Average processing kernel time (high-res timer):  0.8530562 sec.
Compute percentage of runtime:                       99.9952297
Compute percentage simulation time (Advection-only): 92.7623401
I/O percentage of simulation time:                    7.2376599
------------------------------------------------------------------
Ratio Compute time vs. I/O time:              1788.8475113
Ratio Advection time vs. I/O-simulation time: 12.8187770
-----------------------------------------

5

From this benchmark measurement, we observe the following:
- our advect-vs-io ratio is >10: our actual particle simulation is already compute-bound
- our total compute-vs-io ratio is >100: including the kNN search results in a highly compute-bound process overall
- more than 75 percent of the advection is pure calculation, hence the advection is dominated by FLOP's
- 99 percent of the computation time is spent in nearest-neighbour search, which meanns that this would be the one function we need to optimize

Overall, as intended, this case is a very good optimisation example to follow up on. So, now: what can we do to improve the performance ? How can we speed up the calculation ?

## Optimisation 1: Use *NumPy* in own kNN search

In this next step, we use vectorisation and dense memory layout by arranging out particles in tightly-packed memory blocks. This change means we can use optimize hardware technologies, such as *caches*.
How can we do that ? In Python, this is quite simple by glue all operations (as best as possible) of our kNN-function into matrix-operations, that are executed in **NumPy**.

In [3]:
def kNN_particles_NumPy(particles, k=5):
    N = particles.shape[0]
    r = (b-a)
    r = math.sqrt(np.dot(r,r))
    distance_matrix = np.ones((N,N), dtype=np.float64) * r
    for i in range(0, N):
        pt = particles[i, :]
        distance_vector = (particles[:, 0] - pt[0])**2 + (particles[:, 1] - pt[1])**2
        distance_matrix[i, :] = numpy.transpose(distance_vector)
    
    kNN_indices = np.ones((N,k), dtype=np.uint32) * -1
    kNN_distances = np.ones((N,k), dtype=np.float64) * r
    for i in range(0, N):
        for m in range(0, k):
            distance_vector = distance_matrix[i, :]
            j = distance_vector.argmin()
            kNN_indices[i, m] = j
            kNN_distances[i, m] = distance_matrix[i, j]
            distance_matrix[i, j] = r
    return kNN_indices, kNN_distances


# ==== Load flow-field data - measure the time ==== #
s_io_file_time = timer()
f = netcdf.netcdf_file('perlin.nc', 'r')
ftimes = f.variables['time'].data
fX = f.variables['lon'].data
fY = f.variables['lat'].data
fU = f.variables['u'].data
fV = f.variables['v'].data
f.close()
e_io_file_time = timer()
io_file_time = e_io_file_time-s_io_file_time

a = np.array([fX[0], fY[0]])
b = np.array([fX[-1], fY[-1]])
t_0_N = np.array([ftimes[0], ftimes[-1]])
T = t_0_N[1]-t_0_N[0]
dt = 12.0 * 60.0 * 60.0
sim_steps = int(math.floor(T/dt))
total_compute_times = []
advect_compute_time = []
kNN_compute_time = []

sim = Simulation_Benchmark(fX, fY, ftimes, fU, fV)
for i in range(0, 512):
    pt = (b - a) * np.random.random_sample(2)  + a
    sim.add_particle(pt[0], pt[1])

# total_compute_times.clear()
print("==== running the simulation with competing procs ====")
stime_sim = timer()
prev_int_proc = -1
cur_int_proc = 0
while sim.sim_time < sim.sim_end_time:
    stime_compute_advect = timer()
    sim.advect_once(dt)
    etime_compute_advect = timer()
    stime_kNN = timer()
    kNN_particles_NumPy(sim.particles, k=5)
    etime_kNN = timer()
    advectTime = etime_compute_advect-stime_compute_advect
    knnTime = etime_kNN-stime_kNN
    advect_compute_time.append(advectTime)
    kNN_compute_time.append(knnTime)
    total_compute_times.append(advectTime+knnTime)
    proc_done = (sim.sim_time / sim.sim_end_time)*100
    cur_int_proc = int(math.floor(proc_done/10))
    if prev_int_proc != cur_int_proc:
        print("Processing at {:10.7f} percent ...".format(proc_done))
    prev_int_proc = cur_int_proc

etime_sim = timer()
sim_total_time = etime_sim-stime_sim
print("Total processing time (high-res timer): {:10.7f} sec.".format(sim_total_time))
avg_ptimer_hr = np.array(total_compute_times).mean()
print("Average processing kernel time (high-res timer): {:10.7f} sec.".format(avg_ptimer_hr))

total_compute_time = np.array(total_compute_times).sum()
total_advect_time = np.array(advect_compute_time).sum()
total_knn_time = np.array(kNN_compute_time).sum()

compute_to_total_ratio = total_compute_time / (sim_total_time+io_file_time)
compute_to_iototal = total_compute_time / (sim.io_mem_time+io_file_time)
io_to_sim_ratio = sim.io_mem_time / (sim.compute_time+sim.io_mem_time)
advect_to_sim = sim.compute_time / (sim.compute_time+sim.io_mem_time)
advect_to_iosim = sim.compute_time / sim.io_mem_time+io_file_time
advect_compute_percentage = (total_advect_time / total_compute_time) * 100.0
knn_compute_percentage = (total_knn_time / total_compute_time) * 100.0

total_compute_time_Numpy = total_compute_time
total_advect_time_Numpy = total_advect_time
total_knn_time_Numpy = total_knn_time

print("Compute percentage of runtime:                       {:10.7f}".format(compute_to_total_ratio*100.0))
print("Compute percentage simulation time (Advection-only): {:10.7f}".format(advect_to_sim*100.0))
print("I/O percentage of simulation time:                   {:10.7f}".format(io_to_sim_ratio*100.0))
print("------------------------------------------------------------------")
print("Ratio Compute time vs. I/O time:              {:10.7f}".format(compute_to_iototal))
print("Ratio Advection time vs. I/O-simulation time: {:10.7f}".format(advect_to_iosim))
print("------------------------------------------------------------------")
print("Percentage of Advection on Compute time:         {:10.4f}".format(advect_compute_percentage))
print("Percentage of Nearest-Neighbour on Compute time: {:10.4f}".format(knn_compute_percentage))
del sim
del ftimes
del fX
del fY
del fU
del fV
del total_compute_times
del advect_compute_time
del kNN_compute_time
gc.collect()

/home/ckehl/.local/lib/python3.6/site-packages/scipy/io/netcdf.py:314: RuntimeWarning: Cannot close a netcdf_file opened with mmap=True, when netcdf_variables or arrays referring to its data still exist. All data arrays obtained from such files refer directly to data on disk, and must be copied before the file can be cleanly closed. (See netcdf_file docstring for more information on mmap.)
  ), category=RuntimeWarning)


==== running the simulation with competing procs ====
Processing at  0.1351351 percent ...
Processing at 10.0000000 percent ...
Processing at 20.0000000 percent ...
Processing at 30.0000000 percent ...
Processing at 40.0000000 percent ...
Processing at 50.0000000 percent ...
Processing at 60.0000000 percent ...
Processing at 70.0000000 percent ...
Processing at 80.0000000 percent ...
Processing at 90.0000000 percent ...
Processing at 100.0000000 percent ...
Total processing time (high-res timer): 21.9434084 sec.
Average processing kernel time (high-res timer):  0.0296098 sec.
Compute percentage of runtime:                       99.8504835
Compute percentage simulation time (Advection-only): 93.1629152
I/O percentage of simulation time:                    6.8370848
------------------------------------------------------------------
Ratio Compute time vs. I/O time:              35.6252213
Ratio Advection time vs. I/O-simulation time: 13.6267686
--------------------------------------------

0

We see that this change made a *huge* difference by cutting the execution time from *>500s* to just *~ 20s* - a speed-up of 25 times!
We also see that this shift is attributed to the kNN search alone: where before *>99%* of the runtime was spent to compute the kNN, we now spend *just above 50%* of the time to do the kNN computation.

Can we do better ?

## Optimisation 2: Use *SciPy* kNN search

Up until now, we followed a simple procedure:

1. compute the distance metric from each point to each other point
2. select the indices of the points with the k shortest distances

This procedure is knows as a *greedy* or *brute-force* algorithm. It works for all cases without prior assumptions on data distribution, dimensionality, point density, and so forth.
It's conceptually trivial (i.e. )not really breaking any 'new ground' here), but it is also *unbearably* slow.

There are smarter algorithm, especially when executing those searches over-and-over again. In this case, organising the element search in a *tree* can lead to drastic improvements.
That algorithm is also known as *kD-Tree*: cluster all data instances (i.e. all entities; all rows of a matrix) according to their *k*-dimensional distance. Important to note here is that 
the *k* of *kD-Tree* has nothing to do with the *k* in *k-nearest neighbour search* - in our example, the *k* of our *kD-Tree* will be 3, whereas the *k* of our *kNN* will be 5.

Using **SciPy** we can speed up both the *brute-force* algorithm as well as compare it to the *kD-Tree algorithm*. Let's see how that goes and how fast that is.

In [6]:
from scipy.spatial import cKDTree
from scipy.spatial.distance import cdist

def kNN_particles_SciPy_kdtree(particles=np.zeros((2,2)), k=5):
    tree = cKDTree(particles)
    kNN_distances, kNN_indices = tree.query(particles, k=k)
    return kNN_indices, kNN_distances

def kNN_particles_SciPy(particles=np.zeros((2,2)), k=5):
    N = particles.shape[0]
    r = (b-a)
    r = math.sqrt(np.dot(r,r))
    distance_matrix = cdist(particles, particles, metric='seuclidean')
    kNN_indices = np.ones((N,k), dtype=np.uint32) * -1
    kNN_distances = np.ones((N,k), dtype=np.float64) * r
    for i in range(0, N):
        for m in range(0, k):
            distance_vector = distance_matrix[i, :]
            j = distance_vector.argmin()
            kNN_indices[i, m] = j
            kNN_distances[i, m] = distance_matrix[i, j]
            distance_matrix[i, j] = r
    return kNN_indices, kNN_distances

In [7]:
# ==== Load flow-field data - measure the time ==== #
s_io_file_time = timer()
f = netcdf.netcdf_file('perlin.nc', 'r')
ftimes = f.variables['time'].data
fX = f.variables['lon'].data
fY = f.variables['lat'].data
fU = f.variables['u'].data
fV = f.variables['v'].data
f.close()
e_io_file_time = timer()
io_file_time = e_io_file_time-s_io_file_time

a = np.array([fX[0], fY[0]])
b = np.array([fX[-1], fY[-1]])
t_0_N = np.array([ftimes[0], ftimes[-1]])
T = t_0_N[1]-t_0_N[0]
dt = 12.0 * 60.0 * 60.0
sim_steps = int(math.floor(T/dt))
total_compute_times = []
advect_compute_time = []
kNN_compute_time = []

sim = Simulation_Benchmark(fX, fY, ftimes, fU, fV)
for i in range(0, 512):
    pt = (b - a) * np.random.random_sample(2)  + a
    sim.add_particle(pt[0], pt[1])

# total_compute_times.clear()
print("==== running the simulation with competing procs ====")
stime_sim = timer()
prev_int_proc = -1
cur_int_proc = 0
while sim.sim_time < sim.sim_end_time:
    stime_compute_advect = timer()
    sim.advect_once(dt)
    etime_compute_advect = timer()
    stime_kNN = timer()
    kNN_particles_SciPy(sim.particles, k=5)
    etime_kNN = timer()
    advectTime = etime_compute_advect-stime_compute_advect
    knnTime = etime_kNN-stime_kNN
    advect_compute_time.append(advectTime)
    kNN_compute_time.append(knnTime)
    total_compute_times.append(advectTime+knnTime)
    proc_done = (sim.sim_time / sim.sim_end_time)*100
    cur_int_proc = int(math.floor(proc_done/10))
    if prev_int_proc != cur_int_proc:
        print("Processing at {:10.7f} percent ...".format(proc_done))
    prev_int_proc = cur_int_proc

etime_sim = timer()
sim_total_time = etime_sim-stime_sim
print("Total processing time (high-res timer): {:10.7f} sec.".format(sim_total_time))
avg_ptimer_hr = np.array(total_compute_times).mean()
print("Average processing kernel time (high-res timer): {:10.7f} sec.".format(avg_ptimer_hr))

total_compute_time = np.array(total_compute_times).sum()
total_advect_time = np.array(advect_compute_time).sum()
total_knn_time = np.array(kNN_compute_time).sum()

compute_to_total_ratio = total_compute_time / (sim_total_time+io_file_time)
compute_to_iototal = total_compute_time / (sim.io_mem_time+io_file_time)
io_to_sim_ratio = sim.io_mem_time / (sim.compute_time+sim.io_mem_time)
advect_to_sim = sim.compute_time / (sim.compute_time+sim.io_mem_time)
advect_to_iosim = sim.compute_time / sim.io_mem_time+io_file_time
advect_compute_percentage = (total_advect_time / total_compute_time) * 100.0
knn_compute_percentage = (total_knn_time / total_compute_time) * 100.0

total_compute_time_SciPy_512 = total_compute_time
total_advect_time_SciPy_512 = total_advect_time
total_knn_time_SciPy_512 = total_knn_time

print("==== Compute particle-kNN via SciPy using the brute-force algorithm ====")
print("Compute percentage of runtime:                       {:10.7f}".format(compute_to_total_ratio*100.0))
print("Compute percentage simulation time (Advection-only): {:10.7f}".format(advect_to_sim*100.0))
print("I/O percentage of simulation time:                   {:10.7f}".format(io_to_sim_ratio*100.0))
print("------------------------------------------------------------------")
print("Ratio Compute time vs. I/O time:              {:10.7f}".format(compute_to_iototal))
print("Ratio Advection time vs. I/O-simulation time: {:10.7f}".format(advect_to_iosim))
print("------------------------------------------------------------------")
print("Percentage of Advection on Compute time:         {:10.4f}".format(advect_compute_percentage))
print("Percentage of Nearest-Neighbour on Compute time: {:10.4f}".format(knn_compute_percentage))
del sim
del ftimes
del fX
del fY
del fU
del fV
del total_compute_times
del advect_compute_time
del kNN_compute_time
gc.collect()

/home/ckehl/.local/lib/python3.6/site-packages/scipy/io/netcdf.py:314: RuntimeWarning: Cannot close a netcdf_file opened with mmap=True, when netcdf_variables or arrays referring to its data still exist. All data arrays obtained from such files refer directly to data on disk, and must be copied before the file can be cleanly closed. (See netcdf_file docstring for more information on mmap.)
  ), category=RuntimeWarning)


==== running the simulation with competing procs ====
Processing at  0.1351351 percent ...
Processing at 10.0000000 percent ...
Processing at 20.0000000 percent ...
Processing at 30.0000000 percent ...
Processing at 40.0000000 percent ...
Processing at 50.0000000 percent ...
Processing at 60.0000000 percent ...
Processing at 70.0000000 percent ...
Processing at 80.0000000 percent ...
Processing at 90.0000000 percent ...
Processing at 100.0000000 percent ...
Total processing time (high-res timer): 13.5994487 sec.
Average processing kernel time (high-res timer):  0.0183343 sec.
==== Compute particle-kNN via SciPy using the brute-force algorithm ====
Compute percentage of runtime:                       99.7590180
Compute percentage simulation time (Advection-only): 92.8156144
I/O percentage of simulation time:                    7.1843856
------------------------------------------------------------------
Ratio Compute time vs. I/O time:              24.6091038
Ratio Advection time vs. I/O

0

In [8]:
# ==== Load flow-field data - measure the time ==== #
s_io_file_time = timer()
f = netcdf.netcdf_file('perlin.nc', 'r')
ftimes = f.variables['time'].data
fX = f.variables['lon'].data
fY = f.variables['lat'].data
fU = f.variables['u'].data
fV = f.variables['v'].data
f.close()
e_io_file_time = timer()
io_file_time = e_io_file_time-s_io_file_time

a = np.array([fX[0], fY[0]])
b = np.array([fX[-1], fY[-1]])
t_0_N = np.array([ftimes[0], ftimes[-1]])
T = t_0_N[1]-t_0_N[0]
dt = 12.0 * 60.0 * 60.0
sim_steps = int(math.floor(T/dt))
total_compute_times = []
advect_compute_time = []
kNN_compute_time = []

sim = Simulation_Benchmark(fX, fY, ftimes, fU, fV)
for i in range(0, 512):
    pt = (b - a) * np.random.random_sample(2)  + a
    sim.add_particle(pt[0], pt[1])

# total_compute_times.clear()
print("==== running the simulation with competing procs ====")
stime_sim = timer()
prev_int_proc = -1
cur_int_proc = 0
while sim.sim_time < sim.sim_end_time:
    stime_compute_advect = timer()
    sim.advect_once(dt)
    etime_compute_advect = timer()
    stime_kNN = timer()
    kNN_particles_SciPy_kdtree(sim.particles, k=5)
    etime_kNN = timer()
    advectTime = etime_compute_advect-stime_compute_advect
    knnTime = etime_kNN-stime_kNN
    advect_compute_time.append(advectTime)
    kNN_compute_time.append(knnTime)
    total_compute_times.append(advectTime+knnTime)
    proc_done = (sim.sim_time / sim.sim_end_time)*100
    cur_int_proc = int(math.floor(proc_done/10))
    if prev_int_proc != cur_int_proc:
        print("Processing at {:10.7f} percent ...".format(proc_done))
    prev_int_proc = cur_int_proc

etime_sim = timer()
sim_total_time = etime_sim-stime_sim
print("Total processing time (high-res timer): {:10.7f} sec.".format(sim_total_time))
avg_ptimer_hr = np.array(total_compute_times).mean()
print("Average processing kernel time (high-res timer): {:10.7f} sec.".format(avg_ptimer_hr))

total_compute_time = np.array(total_compute_times).sum()
total_advect_time = np.array(advect_compute_time).sum()
total_knn_time = np.array(kNN_compute_time).sum()

compute_to_total_ratio = total_compute_time / (sim_total_time+io_file_time)
compute_to_iototal = total_compute_time / (sim.io_mem_time+io_file_time)
io_to_sim_ratio = sim.io_mem_time / (sim.compute_time+sim.io_mem_time)
advect_to_sim = sim.compute_time / (sim.compute_time+sim.io_mem_time)
advect_to_iosim = sim.compute_time / sim.io_mem_time+io_file_time
advect_compute_percentage = (total_advect_time / total_compute_time) * 100.0
knn_compute_percentage = (total_knn_time / total_compute_time) * 100.0

total_compute_time_SciPy_kd512 = total_compute_time
total_advect_time_SciPy_kd512 = total_advect_time
total_knn_time_SciPy_kd512 = total_knn_time

print("==== Compute particle-kNN via SciPy using the kD-Tree algorithm ====")
print("Compute percentage of runtime:                       {:10.7f}".format(compute_to_total_ratio*100.0))
print("Compute percentage simulation time (Advection-only): {:10.7f}".format(advect_to_sim*100.0))
print("I/O percentage of simulation time:                   {:10.7f}".format(io_to_sim_ratio*100.0))
print("------------------------------------------------------------------")
print("Ratio Compute time vs. I/O time:              {:10.7f}".format(compute_to_iototal))
print("Ratio Advection time vs. I/O-simulation time: {:10.7f}".format(advect_to_iosim))
print("------------------------------------------------------------------")
print("Percentage of Advection on Compute time:         {:10.4f}".format(advect_compute_percentage))
print("Percentage of Nearest-Neighbour on Compute time: {:10.4f}".format(knn_compute_percentage))
del sim
del ftimes
del fX
del fY
del fU
del fV
del total_compute_times
del advect_compute_time
del kNN_compute_time
gc.collect()

/home/ckehl/.local/lib/python3.6/site-packages/scipy/io/netcdf.py:314: RuntimeWarning: Cannot close a netcdf_file opened with mmap=True, when netcdf_variables or arrays referring to its data still exist. All data arrays obtained from such files refer directly to data on disk, and must be copied before the file can be cleanly closed. (See netcdf_file docstring for more information on mmap.)
  ), category=RuntimeWarning)


==== running the simulation with competing procs ====
Processing at  0.1351351 percent ...
Processing at 10.0000000 percent ...
Processing at 20.0000000 percent ...
Processing at 30.0000000 percent ...
Processing at 40.0000000 percent ...
Processing at 50.0000000 percent ...
Processing at 60.0000000 percent ...
Processing at 70.0000000 percent ...
Processing at 80.0000000 percent ...
Processing at 90.0000000 percent ...
Processing at 100.0000000 percent ...
Total processing time (high-res timer):  8.2945392 sec.
Average processing kernel time (high-res timer):  0.0111754 sec.
==== Compute particle-kNN via SciPy using the kD-Tree algorithm ====
Compute percentage of runtime:                       99.6942595
Compute percentage simulation time (Advection-only): 93.2359472
I/O percentage of simulation time:                    6.7640528
------------------------------------------------------------------
Ratio Compute time vs. I/O time:              16.7245541
Ratio Advection time vs. I/O-sim

0

When observing and *interpreting* our measured results, it's important to note here that *SciPy* is a nice toolbox on top of NumPy that includes some 
specific, well know algorithms for geometry (e.g. nearest-neighbours), statistics, inter- and extrapolation and many more. There are other, domain-specific, comparable package (just like *SciPy*) sitting on top of NumPy, such as **scikits-learn** (sklearn) or **scikits-image**.
If the specific function you search for is in that toolbox, and all your other data are in **_NumPy_**, then it's usually good to use those packages instead of re-inventing the wheel with JIT or C-Bindings.
That said, the SciPy-trick doesn't work all the time - some algorithms are just not available in SciPy, or just not convertable or expressable in linear-algebra form.
In order to execute those functions *at high speed* and *inside Python*, we will need to use specific C-bindings, such as provide via **JIT**.

## Optimisation 3: Write the whole kNN search in *Numba*'s Just-in-Time (JIT) C-code

What is *Just-in-Time (JIT)* compilation ? (What is *compilation* ?)

As said before: the computer CPU, in the very end, only understands C-like code (or, more specifically: C-compiled binary code).
Python itself is somehow build with C - you can see this because *python* itself is just an executable program.
Python can interface pre-compiled C-code with its Python translation natively (more to that later). Another possibility is to write code in **_native_** Python, which is translated into C-code during Python-code execution.
This translation of Python code into C-code *when running* the script is called *Just-in-Time (JIT)* compilation (in contrast to the more common *pre-compiled binaries*).

Why would we want to do that ?
Can I transform *any* piece of Python code into fast C-code via JIT compilation ? **NO**

Coding *JIT* snippets is constrained by certain conditions:
- JIT only wraps *individual functions* or full *classes* (latter is more difficult), so you cannot just JIT-compile a few code-lines by choice
- JIT only accepts function inputs from either *native Python* or *NumPy* -> if you have other classes, objects or packages as input, it will not work; if the parameter is a function handle, it will not work;
- Because the JIT does a translation to C, you need to (a) know the types of our parameter variables, (b) keep these variables fixed and (c) do not mix fields of different types
- with JIT, you loose all typeless coding stuff in the target function
- the code inside the JIT function can only use *functions*, *data types* and *instructions* that are native in either (a) native Python or (b) NumPy
- you cannot use other packages in JIT, such as *SciPy*, *sklearn* or others, unless they are declared inside the JIT context

Due to those constraints, producing working JIT-code is actually not *that* easy unless you know the function you want to 'jittify' inside-out.

How can we actually use JIT code ? 1. way (modern): **Numba**

Our starting point of the JIT code is, due to the above-listed constraints, the Python-only implementation from the very beginning.

In [9]:
import numba

@numba.jit("void(float32[:,:], int32[:,:], float64[:,:], int32)", nopython=True)
def kNN_particles_Numba(particles, kNN_indices, kNN_distances, k=5):
    N = particles.shape[0]
    r = (b-a)
    r = math.sqrt(np.dot(r,r))
    distance_matrix = np.ones((N,N), dtype=np.float64) * r
    for i in range(0, N):
        for j in range (i, N):
            if i==j:
                distance_matrix[i,j] = .0
                continue
            dv = particles[i, 0:1]-particles[j, 0:1]
            dv_len_sqr = dv[0]*dv[0]+dv[1]*dv[1]
            distance_matrix[i, j] = dv_len_sqr
            distance_matrix[j, i] = dv_len_sqr

    for i in range(0, N):
        for j in range (0, N):
            if i==j:
                continue
            m = 0
            while (m<k) and ((kNN_indices[i, m] < 0) or (kNN_distances[i, m] <= distance_matrix[i, j])):
                m += 1
            if not m<k:
                continue
            l = k-1
            while l>m:
                kNN_indices[i, l] = kNN_indices[i, l-1]
                kNN_distances[i, l] = kNN_distances[i, l-1]
            kNN_indices[i, m] = j
            kNN_distances[i, m] = distance_matrix[i, j]
    return


# ==== Load flow-field data - measure the time ==== #
s_io_file_time = timer()
f = netcdf.netcdf_file('perlin.nc', 'r')
ftimes = f.variables['time'].data
fX = f.variables['lon'].data
fY = f.variables['lat'].data
fU = f.variables['u'].data
fV = f.variables['v'].data
f.close()
e_io_file_time = timer()
io_file_time = e_io_file_time-s_io_file_time

a = np.array([fX[0], fY[0]])
b = np.array([fX[-1], fY[-1]])
t_0_N = np.array([ftimes[0], ftimes[-1]])
T = t_0_N[1]-t_0_N[0]
dt = 12.0 * 60.0 * 60.0
sim_steps = int(math.floor(T/dt))
total_compute_times = []
advect_compute_time = []
kNN_compute_time = []

sim = Simulation_Benchmark(fX, fY, ftimes, fU, fV)
for i in range(0, 512):
    pt = (b - a) * np.random.random_sample(2)  + a
    sim.add_particle(pt[0], pt[1])

# total_compute_times.clear()
print("==== running the simulation with competing procs ====")
rv = (b-a)
r = math.sqrt(np.dot(rv,rv))
N = len(sim)
k = 5
kNN_indices = np.ones((N,k), dtype=np.int32) * -1
kNN_distances = np.ones((N,k), dtype=np.float64) * r
stime_sim = timer()
prev_int_proc = -1
cur_int_proc = 0
while sim.sim_time < sim.sim_end_time:
    stime_compute_advect = timer()
    sim.advect_once(dt)
    etime_compute_advect = timer()
    stime_kNN = timer()
    kNN_distances.fill(r)
    kNN_indices.fill(np.int32(-1))
    kNN_particles_Numba(sim.particles, kNN_indices, kNN_distances, k)
    etime_kNN = timer()
    advectTime = etime_compute_advect-stime_compute_advect
    knnTime = etime_kNN-stime_kNN
    advect_compute_time.append(advectTime)
    kNN_compute_time.append(knnTime)
    total_compute_times.append(advectTime+knnTime)
    proc_done = (sim.sim_time / sim.sim_end_time)*100
    cur_int_proc = int(math.floor(proc_done/10))
    if prev_int_proc != cur_int_proc:
        print("Processing at {:10.7f} percent ...".format(proc_done))
    prev_int_proc = cur_int_proc

etime_sim = timer()
sim_total_time = etime_sim-stime_sim
print("Total processing time (high-res timer): {:10.7f} sec.".format(sim_total_time))
avg_ptimer_hr = np.array(total_compute_times).mean()
print("Average processing kernel time (high-res timer): {:10.7f} sec.".format(avg_ptimer_hr))

total_compute_time = np.array(total_compute_times).sum()
total_advect_time = np.array(advect_compute_time).sum()
total_knn_time = np.array(kNN_compute_time).sum()

compute_to_total_ratio = total_compute_time / (sim_total_time+io_file_time)
compute_to_iototal = total_compute_time / (sim.io_mem_time+io_file_time)
io_to_sim_ratio = sim.io_mem_time / (sim.compute_time+sim.io_mem_time)
advect_to_sim = sim.compute_time / (sim.compute_time+sim.io_mem_time)
advect_to_iosim = sim.compute_time / sim.io_mem_time+io_file_time
advect_compute_percentage = (total_advect_time / total_compute_time) * 100.0
knn_compute_percentage = (total_knn_time / total_compute_time) * 100.0

total_compute_time_Numba_512 = total_compute_time
total_advect_time_Numba_512 = total_advect_time
total_knn_time_Numba_512 = total_knn_time

print("Compute percentage of runtime:                       {:10.7f}".format(compute_to_total_ratio*100.0))
print("Compute percentage simulation time (Advection-only): {:10.7f}".format(advect_to_sim*100.0))
print("I/O percentage of simulation time:                   {:10.7f}".format(io_to_sim_ratio*100.0))
print("------------------------------------------------------------------")
print("Ratio Compute time vs. I/O time:              {:10.7f}".format(compute_to_iototal))
print("Ratio Advection time vs. I/O-simulation time: {:10.7f}".format(advect_to_iosim))
print("------------------------------------------------------------------")
print("Percentage of Advection on Compute time:         {:10.4f}".format(advect_compute_percentage))
print("Percentage of Nearest-Neighbour on Compute time: {:10.4f}".format(knn_compute_percentage))
del sim
del ftimes
del fX
del fY
del fU
del fV
del total_compute_times
del advect_compute_time
del kNN_compute_time
del kNN_indices
del kNN_distances
gc.collect()

/home/ckehl/.local/lib/python3.6/site-packages/scipy/io/netcdf.py:314: RuntimeWarning: Cannot close a netcdf_file opened with mmap=True, when netcdf_variables or arrays referring to its data still exist. All data arrays obtained from such files refer directly to data on disk, and must be copied before the file can be cleanly closed. (See netcdf_file docstring for more information on mmap.)
  ), category=RuntimeWarning)


==== running the simulation with competing procs ====
Processing at  0.1351351 percent ...
Processing at 10.0000000 percent ...
Processing at 20.0000000 percent ...
Processing at 30.0000000 percent ...
Processing at 40.0000000 percent ...
Processing at 50.0000000 percent ...
Processing at 60.0000000 percent ...
Processing at 70.0000000 percent ...
Processing at 80.0000000 percent ...
Processing at 90.0000000 percent ...
Processing at 100.0000000 percent ...
Total processing time (high-res timer): 22.7322796 sec.
Average processing kernel time (high-res timer):  0.0306646 sec.
Compute percentage of runtime:                       99.8168069
Compute percentage simulation time (Advection-only): 92.5641894
I/O percentage of simulation time:                    7.4358106
------------------------------------------------------------------
Ratio Compute time vs. I/O time:              37.9409767
Ratio Advection time vs. I/O-simulation time: 12.4496398
--------------------------------------------

22836

We observe the following:
- JIT is on-par with the NumPy implementation - that's quite a common result for small functions and code section
- JIT is slower than SciPy (or, at least: the kD-Tree-based algorithm)

On the latter observation point: the data size (here: 512 particles) of our example is tiny, so that tree-construction and distance computation is simple. 
If we were to increase *N = 2048* or larger, we will start to see a reversal in the measurements. 
For very large (i.e. **_big_**) data, the kD-Tree algorithm in SciPy is slower than greedy algorithm, if the particle positions change and hence the tree needs to be revuild at each iteration.

In [10]:
# ==== Load flow-field data - measure the time ==== #
s_io_file_time = timer()
f = netcdf.netcdf_file('perlin.nc', 'r')
ftimes = f.variables['time'].data
fX = f.variables['lon'].data
fY = f.variables['lat'].data
fU = f.variables['u'].data
fV = f.variables['v'].data
f.close()
e_io_file_time = timer()
io_file_time = e_io_file_time-s_io_file_time

a = np.array([fX[0], fY[0]])
b = np.array([fX[-1], fY[-1]])
t_0_N = np.array([ftimes[0], ftimes[-1]])
T = t_0_N[1]-t_0_N[0]
dt = 12.0 * 60.0 * 60.0
sim_steps = int(math.floor(T/dt))
total_compute_times_Numba2k = []
total_compute_times_SciPy2k = []
advect_compute_time = []
kNN_compute_time_Numba2k = []
kNN_compute_time_SciPy2k = []

sim = Simulation_Benchmark(fX, fY, ftimes, fU, fV)
for i in range(0, 2048):
    pt = (b - a) * np.random.random_sample(2)  + a
    sim.add_particle(pt[0], pt[1])

# total_compute_times.clear()
print("==== running the simulation with competing procs ====")
rv = (b-a)
r = math.sqrt(np.dot(rv,rv))
N = len(sim)
k = 5
kNN_indices = np.ones((N,k), dtype=np.int32) * -1
kNN_distances = np.ones((N,k), dtype=np.float64) * r
stime_sim = timer()
prev_int_proc = -1
cur_int_proc = 0
while sim.sim_time < sim.sim_end_time:
    stime_compute_advect = timer()
    sim.advect_once(dt)
    etime_compute_advect = timer()
    stime_kNN_Numba2k = timer()
    kNN_distances.fill(r)
    kNN_indices.fill(np.int32(-1))
    kNN_particles_Numba(sim.particles, kNN_indices, kNN_distances, k)
    etime_kNN_Numba2k = timer()
    stime_kNN_SciPy2k = timer()
    kNN_particles_SciPy(sim.particles, k=5)
    etime_kNN_SciPy2k = timer()
    advectTime = etime_compute_advect-stime_compute_advect
    knnTime_Numba2k = etime_kNN_Numba2k-stime_kNN_Numba2k
    knnTime_SciPy2k = etime_kNN_SciPy2k-stime_kNN_SciPy2k
    advect_compute_time.append(advectTime)
    kNN_compute_time_Numba2k.append(knnTime_Numba2k)
    kNN_compute_time_SciPy2k.append(knnTime_SciPy2k)
    total_compute_times_Numba2k.append(advectTime+knnTime_Numba2k)
    total_compute_times_SciPy2k.append(advectTime+knnTime_SciPy2k)
    proc_done = (sim.sim_time / sim.sim_end_time)*100
    cur_int_proc = int(math.floor(proc_done/10))
    if prev_int_proc != cur_int_proc:
        print("Processing at {:10.7f} percent ...".format(proc_done))
    prev_int_proc = cur_int_proc

etime_sim = timer()
sim_total_time = etime_sim-stime_sim
print("Total processing time (high-res timer): {:10.7f} sec.".format(sim_total_time))
# avg_ptimer_hr = np.array(total_compute_times).mean()
# print("Average processing kernel time (high-res timer): {:10.7f} sec.".format(avg_ptimer_hr))

total_compute_time_Numba_2k = np.array(total_compute_times_Numba2k).sum()
total_advect_time_Numba_2k = np.array(advect_compute_time).sum()
total_knn_time_Numba_2k = np.array(kNN_compute_time_Numba2k).sum()

total_compute_time_SciPy_2k = np.array(total_compute_times_SciPy2k).sum()
total_advect_time_SciPy_2k = np.array(advect_compute_time).sum()
total_knn_time_SciPy_2k = np.array(kNN_compute_time_SciPy2k).sum()

advect_compute_percentage_numba = (total_advect_time_Numba_2k / total_compute_time_Numba_2k) * 100.0
advect_compute_percentage_scipy = (total_advect_time_SciPy_2k / total_compute_time_SciPy_2k) * 100.0
knn_compute_percentage_numba = (total_knn_time_Numba_2k / total_compute_time_Numba_2k) * 100.0
knn_compute_percentage_scipy = (total_knn_time_SciPy_2k / total_compute_time_SciPy_2k) * 100.0

print("==== simulation run with 2^11 instead of 2^9 particles ====")

print("Percentage of Advection on Compute time (with Numba): {:10.4f}".format(advect_compute_percentage_numba))
print("Percentage of Nearest-Neighbour on Compute time (with Numba): {:10.4f}".format(knn_compute_percentage_numba))
print("Percentage of Advection on Compute time (with SciPy): {:10.4f}".format(advect_compute_percentage_scipy))
print("Percentage of Nearest-Neighbour on Compute time (with SciPy): {:10.4f}".format(knn_compute_percentage_scipy))
del sim
del ftimes
del fX
del fY
del fU
del fV
del total_compute_times_Numba2k
del total_compute_times_SciPy2k
del advect_compute_time
del kNN_compute_time_Numba2k
del kNN_compute_time_SciPy2k
del kNN_indices
del kNN_distances
gc.collect()

/home/ckehl/.local/lib/python3.6/site-packages/scipy/io/netcdf.py:314: RuntimeWarning: Cannot close a netcdf_file opened with mmap=True, when netcdf_variables or arrays referring to its data still exist. All data arrays obtained from such files refer directly to data on disk, and must be copied before the file can be cleanly closed. (See netcdf_file docstring for more information on mmap.)
  ), category=RuntimeWarning)


==== running the simulation with competing procs ====
Processing at  0.1351351 percent ...
Processing at 10.0000000 percent ...
Processing at 20.0000000 percent ...
Processing at 30.0000000 percent ...
Processing at 40.0000000 percent ...
Processing at 50.0000000 percent ...
Processing at 60.0000000 percent ...
Processing at 70.0000000 percent ...
Processing at 80.0000000 percent ...
Processing at 90.0000000 percent ...
Processing at 100.0000000 percent ...
Total processing time (high-res timer): 191.4964308 sec.
==== simulation run with 2^11 instead of 2^9 particles ====
Percentage of Advection on Compute time (with Numba):     9.3665
Percentage of Nearest-Neighbour on Compute time (with Numba):    90.6335
Percentage of Advection on Compute time (with SciPy):    34.1590
Percentage of Nearest-Neighbour on Compute time (with SciPy):    65.8410


0

## Optimisation 4: Write the whole kNN search in native C-code and use *ctypes* JIT

A 2. way, and slightly older option, to do JIT is the use of the **ctypes** package. For everyone who is familiar with OceanParcels: This is what is actually in use in OceanParcels.
The idea of JIT is the same, but *ctypes* actually requires the C-function to be written by the programmer, as there is no simple, automatic translation from *Python* to *C*.
The usage and calling convention of *ctypes* is more cumbersome than *Numba*. The advantage of *ctypes* is that it's easy to mix and incorporate more complex, library-interlinked functionality into the JIT-functions than JUST native Python and NumPy.

In our example, the C-code is in separate files called `knn_ctypes.h` and `knn_ctypes.c`. Additionally, I created a few super-classes to make the *ctypes*-calling less verbose and more easy-access.

In [11]:
import ctypes
from code_interface import *
from ctypes import byref
c_lib_register = LibraryRegisterC()
# ========================= #
# 1. Setup and Compilation  #
# ========================= #
c_lib_register.load("knn_ctypes", src_dir="/var/scratch/workspace/Python")  # the string here is the name of the C-code/header file
c_lib_register.register("knn_ctypes")
knn_ctypes_interface = c_lib_register.get("knn_ctypes")  # ["node"]
func_params = []
func_params.append({"name": "get_index_columnorder", "return": ctypes.c_ulong, "arguments": [ctypes.c_int, ctypes.c_int, ctypes.c_int, ctypes.c_int]})
func_params.append({"name": "get_index_roworder", "return": ctypes.c_ulong, "arguments": [ctypes.c_int, ctypes.c_int, ctypes.c_int, ctypes.c_int]})
func_params.append({"name": "kNN_particles_ctypes", "return": None, "arguments": [np.ctypeslib.ndpointer(dtype=np.float32,ndim=2,flags='C_CONTIGUOUS'),  # ctypes.POINTER(np.ctypeslib.as_ctypes_type(np.float64)), 
                                                                                  ctypes.c_int, 
                                                                                  ctypes.c_int, 
                                                                                  np.ctypeslib.ndpointer(dtype=np.int32,ndim=2,flags='C_CONTIGUOUS'),  # ctypes.POINTER(np.ctypeslib.as_ctypes_type(np.int32)), 
                                                                                  np.ctypeslib.ndpointer(dtype=np.float64,ndim=2,flags='C_CONTIGUOUS'),  # ctypes.POINTER(np.ctypeslib.as_ctypes_type(np.float64)), 
                                                                                  ctypes.c_float, 
                                                                                  ctypes.c_int]})
c_funcs = knn_ctypes_interface.load_functions(func_params)
kNN_particles_ctypes = c_funcs["kNN_particles_ctypes"]

# ========================================== #
# 2. Load flow-field data - measure the time #
# ========================================== #
s_io_file_time = timer()
f = netcdf.netcdf_file('perlin.nc', 'r')
ftimes = f.variables['time'].data
fX = f.variables['lon'].data
fY = f.variables['lat'].data
fU = f.variables['u'].data
fV = f.variables['v'].data
f.close()
e_io_file_time = timer()
io_file_time = e_io_file_time-s_io_file_time

a = np.array([fX[0], fY[0]])
b = np.array([fX[-1], fY[-1]])
t_0_N = np.array([ftimes[0], ftimes[-1]])
T = t_0_N[1]-t_0_N[0]
dt = 12.0 * 60.0 * 60.0
sim_steps = int(math.floor(T/dt))
total_compute_times = []
advect_compute_time = []
kNN_compute_time = []

sim = Simulation_Benchmark(fX, fY, ftimes, fU, fV)
for i in range(0, 512):
    pt = (b - a) * np.random.random_sample(2)  + a
    sim.add_particle(pt[0], pt[1])

#
# 3. Execute simulation and NN-search
#
print("==== running the simulation with competing procs ====")
rv = (b-a)
r = math.sqrt(np.dot(rv,rv))
N = len(sim)
k = 5
kNN_indices = np.ones((N,k), dtype=np.int32) * -1
kNN_distances = np.ones((N,k), dtype=np.float64) * r
stime_sim = timer()
prev_int_proc = -1
cur_int_proc = 0
while sim.sim_time < sim.sim_end_time:
    stime_compute_advect = timer()
    sim.advect_once(dt)
    etime_compute_advect = timer()
    stime_kNN = timer()
    kNN_distances.fill(r)
    kNN_indices.fill(np.int32(-1))
    #particle_c_data = byref(np.ctypeslib.as_ctypes(sim.particles))
    #indices_c_ref = byref(np.ctypeslib.as_ctypes(kNN_indices))
    #distances_c_ref = byref(np.ctypeslib.as_ctypes(kNN_distances))
    #kNN_particles_ctypes(particle_c_data, sim.particles.shape[0], sim.particles.shape[1], indices_c_ref, distances_c_ref, r, k)
    particle_data = sim.particles
    kNN_particles_ctypes(particle_data, particle_data.shape[0], particle_data.shape[1], kNN_indices, kNN_distances, r, k)
    etime_kNN = timer()
    advectTime = etime_compute_advect-stime_compute_advect
    knnTime = etime_kNN-stime_kNN
    advect_compute_time.append(advectTime)
    kNN_compute_time.append(knnTime)
    total_compute_times.append(advectTime+knnTime)
    proc_done = (sim.sim_time / sim.sim_end_time)*100
    cur_int_proc = int(math.floor(proc_done/10))
    if prev_int_proc != cur_int_proc:
        print("Processing at {:10.7f} percent ...".format(proc_done))
    prev_int_proc = cur_int_proc

etime_sim = timer()
sim_total_time = etime_sim-stime_sim
print("Total processing time (high-res timer): {:10.7f} sec.".format(sim_total_time))
avg_ptimer_hr = np.array(total_compute_times).mean()
print("Average processing kernel time (high-res timer): {:10.7f} sec.".format(avg_ptimer_hr))

total_compute_time = np.array(total_compute_times).sum()
total_advect_time = np.array(advect_compute_time).sum()
total_knn_time = np.array(kNN_compute_time).sum()

compute_to_total_ratio = total_compute_time / (sim_total_time+io_file_time)
compute_to_iototal = total_compute_time / (sim.io_mem_time+io_file_time)
io_to_sim_ratio = sim.io_mem_time / (sim.compute_time+sim.io_mem_time)
advect_to_sim = sim.compute_time / (sim.compute_time+sim.io_mem_time)
advect_to_iosim = sim.compute_time / sim.io_mem_time+io_file_time
advect_compute_percentage = (total_advect_time / total_compute_time) * 100.0
knn_compute_percentage = (total_knn_time / total_compute_time) * 100.0

total_compute_time_ctypes = total_compute_time
total_advect_time_ctypes = total_advect_time
total_knn_time_ctypes = total_knn_time

print("Compute percentage of runtime:                       {:10.7f}".format(compute_to_total_ratio*100.0))
print("Compute percentage simulation time (Advection-only): {:10.7f}".format(advect_to_sim*100.0))
print("I/O percentage of simulation time:                   {:10.7f}".format(io_to_sim_ratio*100.0))
print("------------------------------------------------------------------")
print("Ratio Compute time vs. I/O time:              {:10.7f}".format(compute_to_iototal))
print("Ratio Advection time vs. I/O-simulation time: {:10.7f}".format(advect_to_iosim))
print("------------------------------------------------------------------")
print("Percentage of Advection on Compute time:         {:10.4f}".format(advect_compute_percentage))
print("Percentage of Nearest-Neighbour on Compute time: {:10.4f}".format(knn_compute_percentage))

#
# 4. re-register and clean-up
#
c_lib_register.deregister("knn_ctypes")
del sim
del ftimes
del fX
del fY
del fU
del fV
del c_lib_register
del total_compute_times
del advect_compute_time
del kNN_compute_time
del kNN_indices
del kNN_distances
gc.collect()

Compiled /var/scratch/workspace/Python/knn_ctypes.c ==> /tmp/PFL-1000/libknn_ctypes.so
lib '/tmp/PFL-1000/libknn_ctypes.so' register (count: 1)
<CDLL '/tmp/PFL-1000/libknn_ctypes.so', handle 2a4aa40 at 0x7f84bb07af60>
{'name': 'get_index_columnorder', 'return': <class 'ctypes.c_ulong'>, 'arguments': [<class 'ctypes.c_int'>, <class 'ctypes.c_int'>, <class 'ctypes.c_int'>, <class 'ctypes.c_int'>]}
{'name': 'get_index_roworder', 'return': <class 'ctypes.c_ulong'>, 'arguments': [<class 'ctypes.c_int'>, <class 'ctypes.c_int'>, <class 'ctypes.c_int'>, <class 'ctypes.c_int'>]}
{'name': 'kNN_particles_ctypes', 'return': None, 'arguments': [<class 'numpy.ctypeslib.ndpointer_<f4_2d_C_CONTIGUOUS'>, <class 'ctypes.c_int'>, <class 'ctypes.c_int'>, <class 'numpy.ctypeslib.ndpointer_<i4_2d_C_CONTIGUOUS'>, <class 'numpy.ctypeslib.ndpointer_<f8_2d_C_CONTIGUOUS'>, <class 'ctypes.c_float'>, <class 'ctypes.c_int'>]}
==== running the simulation with competing procs ====
Processing at  0.1351351 percent ...

/home/ckehl/.local/lib/python3.6/site-packages/scipy/io/netcdf.py:314: RuntimeWarning: Cannot close a netcdf_file opened with mmap=True, when netcdf_variables or arrays referring to its data still exist. All data arrays obtained from such files refer directly to data on disk, and must be copied before the file can be cleanly closed. (See netcdf_file docstring for more information on mmap.)
  ), category=RuntimeWarning)
Exception ignored in: <bound method LibraryRegisterC.__del__ of <code_interface.LibraryRegisterC object at 0x7f84bb0e6b38>>
Traceback (most recent call last):
  File "/var/scratch/workspace/Python/code_interface.py", line 51, in __del__
    while entry.register_count > 0:
AttributeError: 'str' object has no attribute 'register_count'


702

Our observations are as follows:
- our good, fast implementation of kNN in *native C-code* and its access via *ctypes* is faster than JIT via Numba!
- the *ctypes*-JIT is even comparably-fast to SciPy, and can be one of the fastest ways to calculate this k-nearest neighbouring particle problem
- if the speed advantage is worth the effort is *in the eye of the beholder* and distinctly depends on your (or: the programmer's) abilities to write fast C-code
- if you are neither highly-skilled nor highly-interested in programming, this solution is probably not for you ...

## Optimisation 5: Writing larger code sections in C++ - *SWIG*, *CMake* and *Python*

- we saw that C-interface to provide us with the performance we lack in Python
- various JIT-compilers provide *C-bindings* during runtime
- *Just-in-Time* compilation has its limits:
  - compiling external code takes time itself
  - external dependencies and parallelisation hard to do in JIT
  - open door for: dependency errors, pointer errors, lack of testing, undiscovered bottlenecks ...
- larger projects do not use JIT-compiled binaries, then use pre-compiled binaries
  - ever asked yourself why NumPy / SciPy is so fast ? those are pre-compiled c-bindings to high-speed linear algebra libraries via *pre-compiled binaries*
  - allows full customisation and control
  - allows also full C++ as base language (not only strict-C)
  - well embedded in established build & distribution systems: Autotools, CMake, etc.
- most accessible starting point: SWIG & CMake
- we follow up with an example guideline

### 1. Write your C-code

Header *(knn_swig.hpp)*:
```
#ifndef __KNN_SWIG_HPP
#define __KNN_SWIG_HPP

#include <math.h>
#include <malloc.h>
#include <stdio.h>
#include <string.h>

unsigned long get_index_columnorder(int row, int rows, int column, int columns);
unsigned long get_index_roworder(int row, int rows, int column, int columns);
void kNN_particles_swig(float* particles, int rows, int columns, int* kNN_indices, double* kNN_distances, float r, int k);

#endif
#endif // __KNN_SWIG_HPP
```

Code *(knn_swig.cpp)*:
```
#include "knn_swig.hpp"

unsigned long get_index_columnorder(int row, int rows, int column, int columns) {
    return (row*columns)+column;
}

unsigned long get_index_roworder(int row, int rows, int column, int columns) {
    return (column*rows)+row;
}

void kNN_particles_swig(float* particles, int rows, int columns, int* kNN_indices, double* kNN_distances, float r, int k) {
    int N = rows;
    double dx, dy, dv_len_sqr, m, l;
    double* distance_matrix = (double*)malloc(sizeof(double)*rows*rows);
    memset(distance_matrix, (double)r, sizeof(double)*rows*rows);
    for(int i=0; i<N; i++) {
        for(int j=i; j<N; j++) {
            if(i==j) {
                distance_matrix[get_index_columnorder(i,N,j,N)] = 0.0;
                continue;
            }
            dx = particles[get_index_columnorder(i, N, 0, 3)]-particles[get_index_columnorder(j, N, 0, 3)];
            dy = particles[get_index_columnorder(i, N, 1, 3)]-particles[get_index_columnorder(j, N, 1, 3)];
            dv_len_sqr = dx*dx+dy*dy;
            distance_matrix[get_index_columnorder(i,N,j,N)] = dv_len_sqr;
            distance_matrix[get_index_columnorder(j,N,i,N)] = dv_len_sqr;
        }
    }

    for(int i=0; i<N; i++) {
        for(int j=i; j<N; j++) {
            if(i==j) {
                continue;
            }
            m = 0;
            while((m<k) && ((kNN_indices[get_index_columnorder(i, N, m, k)] < 0) || (kNN_distances[get_index_columnorder(i, N, m, k)] <= distance_matrix[get_index_columnorder(i, N, j, N)]))) {
                m++;
            }
            if(m >= k) {
                continue;
            }
            l = k-1;
            while(l>m) {
                kNN_indices[get_index_columnorder(i, N, l, k)] = kNN_indices[get_index_columnorder(i, N, l-1, k)];
                kNN_distances[get_index_columnorder(i, N, l, k)] = kNN_distances[get_index_columnorder(i, N, l-1, k)];
            }
            kNN_indices[get_index_columnorder(i, N, m, k)] = j;
            kNN_distances[get_index_columnorder(i, N, m, k)] = distance_matrix[get_index_columnorder(i, N, j, N)];
        }
    }


    free(distance_matrix);
    return;
}
```

### 2. Write your CMakeFiles and folder structure for your C-code project

```
knn_cmake
- CMake
  - FindPython.cmake
  - FindPython2.cmake
  - FindPython3.cmake
  - FindSWIG.cmake
  - FindNumpy.cmake
- include
  - knn_swig.hpp
- src
  - knn_swig.cpp
- wrapping
  - python
    - CMakeLists.txt
    - knn_swig.i
  - CMakeLists.txt
- CMakeLists.txt
```

- How do I write CMake code ? That's a 3-month course in itself ...
- show example

### 3. Write SWIG-Python interface file

Interface *(knn_swig.i)*:
```
%module knn_swig

// Add necessary symbols to generated header
%{
#define SWIG_FILE_WITH_INIT
#include <Python.h>
#include <knn_swig.hpp>
%}

%include "stdint.i"
%include "std_string.i"
%include "typemap.i"
%include "numpy.i"

// Process symbols in header
%include "knn_swig.hpp"

%unignore ""; // unignore all
```

### 4. Run CMake, compile the library

```
cd knn_cmake
mkdir build && cd build
ccmake ..
make
make install
```

### 5. Import the generated wrapper and use the function

in your python code:
```
from knn_swig import *
kNN_particles_ctypes(particles, rows, columns, kNN_indices, kNN_distances, r, k)
```

### Why do I not have a running example ?

- coding the whole CMake structure takes half a week, even for me, even for this simple example
- too much time for a PFL
- lots of effort (perhaps too much) in small, temporary projects or few simulation scenarios
- if you wanna build a fast, efficient and large library for long-term, the time investment does pay off
- side-note: As I am mainly a computer scientist involved with speed, graphics, visualisation and the like, using *SWIG*, *CMake* and *C++* is my starndard *modus operandus*
  - gives full control
  - provides the fastest code (because it really runs actually C++ code, you just don't see it as a user)
  - if facilitates interfaces to the whole software library database of the past 65 years
  - it has virtually no limitations - APART from having very long development times
- development time with C++, SWIG, CMake and Python
  - it is long - really, really long 
    - if you need to get something to show in a week, this 'toolchain' is not your choice
    - if you use this toolchain, requests for short-term changes are 'neyed' by-default - on purpose and for good reasons
    - this style of coding requires long-term planning, vision and dedication

## Conclusion - Comparison of different computational modes and C-bindings

Lastly, we collect the measurements again from the different performance optimisations to gain a holistic view on the benchmarking and the achieve speed-ups.

In [12]:
advect_measurements = []
advect_measurements.append(total_advect_time_PythonOnly)
advect_measurements.append(total_advect_time_Numpy)
advect_measurements.append(total_advect_time_SciPy_512)
advect_measurements.append(total_advect_time_Numba_512)
advect_measurements.append(total_advect_time_SciPy_kd512)
advect_measurements.append(total_advect_time_ctypes)
advects_npa = np.array(advect_measurements)
print("Advection - mean runtime and standard variance:                  {:4.4f} sec. ({:4.6f} sec)".format(advects_npa.mean(), advects_npa.std()))
print("==== kNN runtimes - implementation comparison ====")
print("k-nearest neighbour search (NxN search space) - plain Python:    {:4.4f} sec.".format(total_knn_time_PythonOnly))
print("k-nearest neighbour search (NxN search space) - NumPy:           {:4.4f} sec.".format(total_knn_time_Numpy))
print("k-nearest neighbour search (NxN search space) - SciPy:           {:4.4f} sec.".format(total_knn_time_SciPy_512))
print("k-nearest neighbour search (NxN search space) - SciPy (kD-Tree): {:4.4f} sec.".format(total_knn_time_SciPy_kd512))
print("k-nearest neighbour search (NxN search space) - Numba:           {:4.4f} sec.".format(total_knn_time_Numba_512))
print("k-nearest neighbour search (NxN search space) - ctypes:          {:4.4f} sec.".format(total_knn_time_ctypes))
print("k-nearest neighbour search (NxN search space) - SciPy (2k pts.): {:4.4f} sec.".format(total_knn_time_SciPy_2k))
print("k-nearest neighbour search (NxN search space) - Numba (2k pts.): {:4.4f} sec.".format(total_knn_time_Numba_2k))
print("==== kNN workload, relative to overall simulation - implementation comparison ====")
print("Workload percentage of kNN of total runtime - plain Python:    {:4.4f} ".format(total_knn_time_PythonOnly*100.0/total_compute_time_PythonOnly))
print("Workload percentage of kNN of total runtime - NumPy:           {:4.4f}".format(total_knn_time_Numpy*100.0/total_compute_time_Numpy))
print("Workload percentage of kNN of total runtime - SciPy:           {:4.4f}".format(total_knn_time_SciPy_512*100.0/total_compute_time_SciPy_512))
print("Workload percentage of kNN of total runtime - SciPy (kD-Tree): {:4.4f}".format(total_knn_time_SciPy_kd512*100.0/total_compute_time_SciPy_kd512))
print("Workload percentage of kNN of total runtime - Numba:           {:4.4f}".format(total_knn_time_Numba_512*100.0/total_compute_time_Numba_512))
print("Workload percentage of kNN of total runtime - ctypes:          {:4.4f}".format(total_knn_time_ctypes*100.0/total_compute_time_ctypes))
print("==== Speed-Up factors - unitless scale factors - relative to 'plain Python' implementation ====")
print("kNN - speed-up - NumPy:           {:6.2f}".format(total_knn_time_PythonOnly / total_knn_time_Numpy))
print("kNN - speed-up - SciPy:           {:6.2f}".format(total_knn_time_PythonOnly / total_knn_time_SciPy_512))
print("kNN - speed-up - SciPy (kD-Tree): {:6.2f}".format(total_knn_time_PythonOnly / total_knn_time_SciPy_kd512))
print("kNN - speed-up - Numba:           {:6.2f}".format(total_knn_time_PythonOnly / total_knn_time_Numba_512))
print("kNN - speed-up - ctypes:          {:6.2f}".format(total_knn_time_PythonOnly / total_knn_time_ctypes))


Advection - mean runtime and standard variance:                  7.6090 sec. (1.319119 sec)
==== kNN runtimes - implementation comparison ====
k-nearest neighbour search (NxN search space) - plain Python:    626.3603 sec.
k-nearest neighbour search (NxN search space) - NumPy:           12.8126 sec.
k-nearest neighbour search (NxN search space) - SciPy:           5.8021 sec.
k-nearest neighbour search (NxN search space) - SciPy (kD-Tree): 0.8666 sec.
k-nearest neighbour search (NxN search space) - Numba:           14.5471 sec.
k-nearest neighbour search (NxN search space) - ctypes:          4.0169 sec.
k-nearest neighbour search (NxN search space) - SciPy (2k pts.): 29.2809 sec.
k-nearest neighbour search (NxN search space) - Numba (2k pts.): 146.9952 sec.
==== kNN workload, relative to overall simulation - implementation comparison ====
Workload percentage of kNN of total runtime - plain Python:    99.2236 
Workload percentage of kNN of total runtime - NumPy:           58.4752
Workload

We can conclude the following statements, when it comes to performance:
- Coding compute-intensive functions in plain Python is *extremely inefficient* use of computational resources - it's very slow
- Significant speed-ups can already be achieved with little effort by cleverly glueing one's physics-problem into matrix-like operations, using **NumPy** and **SciPy**
- *SciPy*'s functions are nice, but 
  - (a) one needs to read up what those individual algorithms and functions do, to see if they apply to one's own scenarion
  - (b) the functions, because they are tied to linear algebra matrix operations, can be very slow for large problem that don't fit into dense-matrix solvers
- **Numba** is a very recent, very modern and quite easy way to transform one's original, simplistic Python-functions into super-fast C-functions with comparably little effort
- you might as 'comparably to what': comparably to **ctypes**, which was the standard for JIT-compilation into C-code from the time before **Numba**; have a look into those helper-functions for the *ctypes* implementation to get an idea how difficult JIT-compilation was before ...
- If you are building a long-lasting, large project that require a large amount of high-density computing and C-operations, then you need to familiarise with *pre-compiled C/C++ libraries* and the tools required to generate them
  - **CMake**: the most widely-used, most user-friendly, but also most-compilcated-to-learn build system on the planet ...
  - **Swig**: a versatile mark-up language to generate pre-compiled wrappers from C/C++ into languages such as Go, Python, Java, Perl and many more
  - actual _**C**_ and **_C++_**

